### 데이터 로드

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms

In [ ]:
# CIFAR-10 데이터셋에 대한 전처리 설정
transform = transforms.Compose([
    transforms.ToTensor(),#이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # 각 텐서의 채널을 정규화함
])

# Load CIFAR-10 training dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Load CIFAR-10 test dataset
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# CIFAR-10 데이터셋으로부터 클래스 레이블 가져오기
class_labels = train_dataset.classes


# 이미지와 레이블을 함께 출력하는 함수
def imshow(images, labels):
    batch_size = images.shape[0]

    fig, axarr = plt.subplots(1, batch_size, figsize=(10, 4))

    for i in range(batch_size):
        img = images[i] / 2 + 0.5  # 언노말라이즈
        npimg = img.numpy()
        axarr[i].imshow(np.transpose(npimg, (1, 2, 0)))
        axarr[i].set_title(class_labels[labels[i].item()])
        axarr[i].axis('off')

    plt.show()

# 학습 데이터의 일부를 가져옴
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
images, labels = next(iter(dataloader))

# 이미지와 레이블을 함께 출력
imshow(images, labels)

### 모델 구성

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolution layer
        self.conv_layer1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = torch.nn.ReLU()
        self.maxpooling1 = torch.nn.MaxPool2d(2)

        self.conv_layer2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = torch.nn.ReLU()
        self.maxpooling2 = torch.nn.MaxPool2d(2)

        # Fully connected layer
        self.fc = torch.nn.Linear(8*8*64, 10)

        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.maxpooling1(self.relu1(self.conv_layer1(x)))
        out = self.maxpooling2(self.relu2(self.conv_layer2(out)))
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolution layer
        self.conv_layer1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = torch.nn.ReLU()
        self.maxpooling1 = torch.nn.MaxPool2d(2)

        self.conv_layer2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = torch.nn.ReLU()
        self.maxpooling2 = torch.nn.MaxPool2d(2)

        # Fully connected layer
        self.fc = torch.nn.Linear(8*8*64, 10)

        torch.nn.init.xavier_uniform_(self.fc.weight)

In [ ]:
self.conv_layer2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
self.relu2 = torch.nn.ReLU()
self.maxpooling2 = torch.nn.MaxPool2d(2)

# Fully connected layer
self.fc = torch.nn.Linear(8*8*64, 10)

In [ ]:
# CNN 모델의 인스턴스 생성
model = CNN()

# 하이퍼파라미터 설정
learning_rate = 0.001

# 손실 함수와 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 훈련 루프
num_epochs = 10
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs, labels = data

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # outputs 계산
        outputs = model(inputs)

        # Loss 계산
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # 옵티마이저 업데이트
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:
            print(f'Epoch {epoch+1}, Batch {i+1}/{len(train_dataloader)}, Loss: {running_loss/100:.4f}')
            running_loss = 0.0

In [4]:
# 테스트 루프
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

total_correct = 0
total_samples = 0

NameError: name 'test_dataset' is not defined

In [ ]:
with torch.no_grad():
    for data in test_dataloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

In [ ]:
accuracy = 100 * total_correct / total_samples
print(f'Test Accuracy: {accuracy:.2f}%')

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), 'model.pth')

# 모델 불러오기
model = CNN()
model.load_state_dict(torch.load('model.pth'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 모델을 평가 모드로 설정
model.eval()

# 테스트 데이터셋에서 쿼리 이미지 선택
query_image_index = 1
query_image, query_label = test_dataset[query_image_index]

# 쿼리 이미지에 대해 추론 수행
with torch.no_grad():
    query_image = query_image.unsqueeze(0)
    query_features = model(query_image)

# 데이터셋 내 다른 이미지들과의 코사인 유사도 계산
similarities = []
for i in range(len(test_dataset)):
    # 데이터셋으로부터 다른 이미지와 해당 레이블 가져오기
    image, _ = test_dataset[i]

    # 이미지 텐서에 배치 차원 추가
    image = image.unsqueeze(0)

    # 모델을 통과하여 특성 추출
    features = model(image)

    # 쿼리 이미지와의 코사인 유사도 계산
    similarity = cosine_similarity(query_features.detach().numpy(), features.detach().numpy())

    # 코사인 유사도와 해당 이미지 인덱스를 similarities 리스트에 저장
    similarities.append((i, similarity.item()))

In [ ]:
# 유사도 값 내림차순 정렬
similarities.sort(key=lambda x: x[1], reverse=True)

# 상위 K개의 이미지 시각화
top_k = 5
fig, axes = plt.subplots(1, top_k, figsize=(6, 3))

for i, (image_index, similarity) in enumerate(similarities[:top_k]):
    image, _ = test_dataset[image_index]

    # 넘파이로 변환
    image_np = image.permute(1, 2, 0).detach().numpy()

    # 이미지 픽셀값 [0, 1]로 스케일링
    image_np = (image_np - np.min(image_np)) / (np.max(image_np) - np.min(image_np))

		# imshow를 위해서 RGB data ([0..1] for floats or [0..255] for integers) 사이의 값으로 정의해야 합니다.

    # subplot으로 시각화
    axes[i].imshow(image_np)
    axes[i].axis('off')
    axes[i].set_title(f'Similarity: {similarity:.4f}', fontsize=8)

plt.tight_layout()
plt.show()